In [11]:
from pyspark import SparkContext
import pandas as pd
from time import time
import re

In [6]:
sc = SparkContext('local')

22/03/27 17:48:12 WARN Utils: Your hostname, Donors-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
22/03/27 17:48:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/27 17:48:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
mis = sc.textFile('miserables.txt')
mis.take(10)
mis.getNumPartitions()
mis.sample(False, 0.1, 81).collect()

['',
 '',
 '',
 ' BOOK FIRST—A JUST MAN',
 ' CHAPTER IV—WORKS CORRESPONDING TO WORDS',
 ' CHAPTER VI—WHO GUARDED HIS HOUSE FOR HIM',
 '',
 '',
 '',
 '',
 '',
 '',
 ' MUNICIPAL POLICE',
 ' CHAPTER I—THE BEGINNING OF REPOSE',
 '',
 ' CHAPTER VI—SISTER SIMPLICE PUT TO THE PROOF',
 '',
 '',
 ' CHAPTER I—WHAT IS MET WITH ON THE WAY FROM NIVELLES',
 '',
 '',
 ' CHAPTER X—THE PLATEAU OF MONT-SAINT-JEAN',
 '',
 '',
 ' CHAPTER XVIII—A RECRUDESCENCE OF DIVINE RIGHT',
 ' CHAPTER XIX—THE BATTLE-FIELD AT NIGHT',
 '',
 '',
 ' CHAPTER VI—WHICH POSSIBLY PROVES BOULATRUELLE’S INTELLIGENCE',
 ' CHAPTER VII—COSETTE SIDE BY SIDE WITH THE STRANGER IN THE DARK',
 ' CHAPTER X—HE WHO SEEKS TO BETTER HIMSELF MAY RENDER HIS SITUATION',
 ' BOOK FOURTH—THE GORBEAU HOVEL',
 ' CHAPTER I—THE ZIGZAGS OF STRATEGY',
 '',
 ' CHAPTER IX—THE MAN WITH THE BELL',
 ' CHAPTER X—WHICH EXPLAINS HOW JAVERT GOT ON THE SCENT',
 '',
 ' CHAPTER IV—GAYETIES',
 ' CHAPTER VI—THE LITTLE CONVENT',
 '',
 ' CHAPTER X—ORIGIN OF THE PERPETUA

In [12]:
miserables_clean = mis.map(lambda x: x.lower())\
    .map(lambda x: re.sub("[.,-]", ' ', x))\
    .map(lambda x: x.replace('’', ' '))

miserables_clean.take(10)

['les misérables',
 '',
 'by victor hugo',
 '',
 '',
 'translated by isabel f  hapgood',
 '',
 '',
 'thomas y  crowell & co ',
 ' no  13  astor place']

In [18]:
miserables_flat = miserables_clean.flatMap(lambda x: x.split(' '))
miserables_flat.take(10)

['les', 'misérables', '', 'by', 'victor', 'hugo', '', '', 'translated', 'by']

In [25]:
mots = miserables_flat.map(lambda x: (x, 1))\
                      .reduceByKey(lambda x, y: x+y)

In [27]:
sorted(mots.collect(), key=lambda x: x[1])

/Users/donor/PycharmProjects/Reviews/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


[('isabel', 1),
 ('hapgood', 1),
 ('crowell', 1),
 ('&', 1),
 ('astor', 1),
 ('copyright', 1),
 ('1887', 1),
 ('spines]', 1),
 ('bookcover]', 1),
 ('frontpapers]', 1),
 ('frontispiece]', 1),
 ('one]', 1),
 ('verso]', 1),
 ('i—fantine', 1),
 ('ix—thénardier', 1),
 ('iv—saint', 1),
 ('v—jean', 1),
 ('illustrations', 1),
 ('bookcover', 1),
 ('frontpapers', 1),
 ('verso', 1),
 ('artificially', 1),
 ('century—the', 1),
 ('crippling', 1),
 ('light—are', 1),
 ('unsolved;', 1),
 ('world;—in', 1),
 ('wider', 1),
 ('destining', 1),
 ('prevalent', 1),
 ('intelligent;', 1),
 ('precipitation;', 1),
 ('magnifying', 1),
 ('—did', 1),
 ('affections', 1),
 ('fortune?', 1),
 ('1804', 1),
 ('b——', 1),
 ('[brignolles]', 1),
 ('curacy—just', 1),
 ('known—took', 1),
 ('parishioners', 1),
 ('—noise', 1),
 ('words—_palabres_', 1),
 ('engross', 1),
 ('curé_', 1),
 ('“respectable”;', 1),
 ('leanness', 1),
 ('maturity;', 1),
 ('sex;', 1),
 ('enclosing', 1),
 ('drooping;—a', 1),
 ('corpulent', 1),
 ('bustling;', 

In [29]:
one_liner = sc.textFile('miserables.txt')\
              .map(lambda x: x.lower())\
              .map(lambda x: re.sub("[.,-]", ' ', x))\
              .map(lambda x: x.replace('’', ' '))\
              .flatMap(lambda x: x.split(' '))\
              .map(lambda x: (x, 1))\
              .reduceByKey(lambda x, y: x+y)\
              .sortBy(lambda couple: couple[1], ascending = True)\
              .collect()


/Users/donor/PycharmProjects/Reviews/venv/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


In [30]:
one_liner

[('isabel', 1),
 ('hapgood', 1),
 ('crowell', 1),
 ('&', 1),
 ('astor', 1),
 ('copyright', 1),
 ('1887', 1),
 ('spines]', 1),
 ('bookcover]', 1),
 ('frontpapers]', 1),
 ('frontispiece]', 1),
 ('one]', 1),
 ('verso]', 1),
 ('i—fantine', 1),
 ('ix—thénardier', 1),
 ('iv—saint', 1),
 ('v—jean', 1),
 ('illustrations', 1),
 ('bookcover', 1),
 ('frontpapers', 1),
 ('verso', 1),
 ('artificially', 1),
 ('century—the', 1),
 ('crippling', 1),
 ('light—are', 1),
 ('unsolved;', 1),
 ('world;—in', 1),
 ('wider', 1),
 ('destining', 1),
 ('prevalent', 1),
 ('intelligent;', 1),
 ('precipitation;', 1),
 ('magnifying', 1),
 ('—did', 1),
 ('affections', 1),
 ('fortune?', 1),
 ('1804', 1),
 ('b——', 1),
 ('[brignolles]', 1),
 ('curacy—just', 1),
 ('known—took', 1),
 ('parishioners', 1),
 ('—noise', 1),
 ('words—_palabres_', 1),
 ('engross', 1),
 ('curé_', 1),
 ('“respectable”;', 1),
 ('leanness', 1),
 ('maturity;', 1),
 ('sex;', 1),
 ('enclosing', 1),
 ('drooping;—a', 1),
 ('corpulent', 1),
 ('bustling;', 

In [32]:
sc.stop()